# Transform pushing

In [6]:
import os
import sys
sys.path.append('../rai-python/build')
import numpy as np
import libry as ry

## Functions

In [7]:
'''
Reads the world file
and returns it as an array
'''
def get_world(folder, idx):
    world_list = []

    file = f'{idx}.txt'

    f = open(folder+file, 'r')
    
    for line in f:
        row = []

        for char in line:
            if char == '\n':
                break

            row.append([int(char)])

        world_list.append(row)


    world = np.asarray(world_list)
    off = -np.floor(world.shape[0] / 2) + 0.5
    
    return world, off

'''
Gets the world as an numeric array
and generates a rai-Config of it
'''
def transform_world(C, world, off, colors):
    pusher = C.addFile('../my-robotModels/cylRobo/robo.g')

    for x in range(world.shape[0]):
        for y in range(world.shape[1]):
            if(world[x][y] == 5):
                continue
            
            empty = C.addFrame(name=f'empty_{x}_{y}', parent='floor', args='type:ssBox size:[1, 1, 1, 0], contact, joint:rigid')
            empty.setPosition([(x+off), (y+off), -0.5])
            empty.setMass(100.0)
            
            if(world[x][y] == 1):
                c = colors[(x+y) % 2]
                block = C.addFrame(name=f'block_{x}_{y}', parent='floor', args=f'type:ssBox size:[1, 1, 1, .02] color=[{c[0]}, {c[1]}, {c[2]}], contact, joint:rigid')
                block.setPosition([(x+off), (y+off), 0.5])
                block.setMass(100.0)

            if(world[x][y] == 2):
                pusher = C.getFrame(frameName='robo')
                pusher.setPosition([(x+off), (y+off), 0.1])

            if(world[x][y] == 3):
                box = C.addFrame(name=f'box_{x}_{y}', parent='floor', args='type:ssBox size:[.9, .9, 1, .02] color=[.6, .3, 0.], contact')
                box.setPosition([(x+off), (y+off), 0.5])
                box.setMass(100.0)
            if(world[x][y] == 4):
                goal = C.addFrame(name=f'goal_{x}_{y}', parent='floor', args='type:ssBox size=[.95, .95, .002, .002] color:[0., 1., 0.], joint:rigid')
                goal.setPosition([(x+off), (y+off), -0.001])


## Do Transformation

In [8]:
# Setup vars
run = 1
SAFE_DIR = f'../agents/maze/pushing_turtle_{run}_log/generated/'
colors = [[.8, .1, .1], [1., .3, .2]]

########


# Get number of Files
n = -1
listdir = os.listdir(SAFE_DIR)
for f in listdir:
    try:
        val = int(f.split('.')[0])
        n = max(n, val)
    except ValueError:
        continue      

# Iterate over all files
for i in range(0, n+1):
    C = ry.Config()
    C.clear()
        
    world, off = get_world(SAFE_DIR, i)       
    transform_world(C, world, off, colors)
        
    g_file = f'{i}.g'
    C.save(SAFE_DIR+g_file)
    C.view()

AttributeError: 'libry.Config' object has no attribute 'save'

## Test

In [7]:
C = ry.Config()
C.clear()

run = 1
file = 0
C.addFile(f'../agents/maze/pushing_turtle_{run}_log/generated/{file}.g')

C.view()